In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import numpy.matlib as ml

In [3]:
A = np.array([[1,0,0,0,0,0],
     [0,2,0,0,0,0],
     [0,0,3,0,0,0],
     [0,0,0,4,0,0],
     [0,0,0,0,5,0],
     [0,0,0,0,0,6],
    ])
x = np.array([[0.3804],
              [0.5678],
              [0.0759],
              [0.0540],
              [0.5308],
              [0.7792]
    ])

m = 4
a = 2.5
b = 5.5
a0 = 1
expected =  [ [0.3804], [0.07060463], [-0.00076315], [0.0005567], [-0.00533705], [0.09689174]]

In [4]:
def chebyshev_filter(A, x,m,a,b,a0):
    e = (b - a)/2
    c = (b + a)/2
    sigma = e/(a0 - c)
    sigma_1 = sigma;
    y = (np.dot(A,x) - c*x)*sigma_1/e
    for i in range (1,m):
        sig_new = 1 / (2/sigma_1 - sigma)
        y_new = 2*(np.dot(A,y) - c*y)*sig_new/e - sigma*sig_new*x
        x = y
        y = y_new
        sigma = sig_new;
    return y

chebyshev_filter_out = chebyshev_filter(A, x,m,a,b,a0)
print(all(abs(chebyshev_filter_out - expected) < 1e-5))

True


In [53]:
B = np.array([[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0]])
display(x, x.shape)
display(x.T, x.T.shape)
display(B)
display(B.T)
display(B[0,:])
display(np.dot(B,x))
display(np.dot(x.T,B))

array([[0.3804],
       [0.5678],
       [0.0759],
       [0.054 ],
       [0.5308],
       [0.7792]])

(6, 1)

array([[0.3804, 0.5678, 0.0759, 0.054 , 0.5308, 0.7792]])

(1, 6)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

array([[1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

array([1, 0, 0, 0, 0, 0])

array([[0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804]])

array([[2.3881, 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [79]:
def cheby_dav_eigh(A, x_vec, m, k_keep, dim_max, tol):
    eig_val = []
    kc = 0
    iter_max = 10#0000

    V = np.zeros(A.shape,dtype=float)
    W = np.zeros(A.shape,dtype=float)
    H = np.zeros(A.shape,dtype=float)
    
    x = x_vec/la.norm(x_vec)
    x_arr = np.array(x)
    V[:,0] = x[:,0]
    w = np.dot(A,x)
    W[:,0] = w[:,0]
    
    mu = np.dot(x.T,w)[0,0]
    H[0,0] = mu

    r = W[:,0] - mu*x
    
    if la.norm(r) <= tol:
        eig_val[0] = mu
        kc = 1
        H = np.array([])
    else:
        kc= 0
    #print('H',H)
    
    #TODO: esimate upper bound of eigenvalues
    upperb = 10
    lowerb = (upperb+mu)/2
    a0 = lowerb
    
    ksub = 0
    iter = 0
    while iter <= iter_max:
        iter+=1
        t = chebyshev_filter(A, x, m, lowerb, upperb, a0)
        
        #orthogonalize t against first k_sub columns of V by householder
        for c in range(ksub):
            v  = V[:,c]
            v = v.reshape(v.size,1)
            t = t - v * np.dot(v.T, t)[0][0]/np.dot(v.T,v)
        t = t/la.norm(t)
        V[:,ksub+1] = t[:,0]
        kold = ksub
        ksub = ksub + 1;
        W[:,ksub] = A @ V [:, ksub]
        H[0 : ksub - kc+1, ksub - kc] = V[:, kc : ksub+1].T @ W[:, ksub]
        H[ksub - kc, 0 : ksub - kc+1] = H[0 : ksub - kc+1, ksub - kc].T
        
    #5. Compute the eigen-decomposition of H: HY = Y D,
    #    where diag(D) is in non-increasing order. Set μ = D(1, 1).
    #6. If (ksub ≥ dimmax) then restart: set ksub = kc + kkeep.
    #7. Update basis: V (:, kc + 1 : ksub) ← V (:, kc + 1 : kold)Y (:, 1 : ksub − kc);
    #    update W: W(:, kc + 1 : ksub) ← W(:, kc + 1 : kold)Y (:, 1 : ksub − kc).
    #8. Compute the residual vector: r = W(:, kc + 1) − μV (:, kc + 1).
    #9. Set noswap = 0, iter ← iter + 1.
    #10. Test for convergence: If norm(r) <= tau * max(diag(D)), set kc = kc + 1,
    #    set eval(kc) = μ; also swap eigenpairs if necessary (see Comment v)
    #    so that converged eigenvalues are in non-increasing order;
    #    set noswap = 1 if any swap happens.
    #11. If (kc ≥ kwant and noswap == 0), Return eval(1 : kc) and V (:, 1 : kc)
    #    as the converged wanted eigenpairs. Exit.
    #12. Update lower bounds: lowerb = median(diag(D));
    #    If a0 > min(diag(D)), set a0 ← min(diag(D)).
    #13. Set the next Ritz vector for filtering: x = V (:, kc + 1).
    #14. Update H: H = D(kc + 1 : ksub, kc + 1 : ksub).
        
x_vec = x
tol = 1e-12
cheby_dav_eigh(A, x_vec, m, 2, 5, tol)

[[4.33897328 1.07134615 0.         0.         0.         0.        ]
 [0.         1.44814798 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]]
[[4.33897328 1.07134615 0.         0.         0.         0.        ]
 [1.07134615 1.44814798 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]]
[[ 4.33897328  1.07134615 -1.89545066  0.          0.          0.        ]
 [ 1.07134615  1.44814798  0.96793031  0.          0.          0.        ]
 [ 0.          0.   

IndexError: index 6 is out of bounds for axis 1 with size 6

In [26]:
B=A
padB = [[0],[0],[0],[0],[0],[0]]
np.pad(B, ((0,0),(0,1)), 'constant', constant_values=0)

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 4, 0, 0, 0],
       [0, 0, 0, 0, 5, 0, 0],
       [0, 0, 0, 0, 0, 6, 0]])

In [55]:

W=np.random.rand(6,2)
print(W)
print(A[:,2:4])
print(A[:,2:4].T @ W)

[[0.80231364 0.51122494]
 [0.55727141 0.54403408]
 [0.98821887 0.34084358]
 [0.2798732  0.66829121]
 [0.93634563 0.19028284]
 [0.96706293 0.30129424]]
[[0 1]
 [0 1]
 [3 0]
 [0 4]
 [0 0]
 [0 0]]
[[2.96465661 1.02253073]
 [2.47907785 3.72842387]]


In [ ]:
A[:,]

In [54]:
B=A
print(B)
B[0:2,3] = np.array([1, 1])
B

[[1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


array([[1, 0, 0, 1, 0, 0],
       [0, 2, 0, 1, 0, 0],
       [0, 0, 3, 0, 0, 0],
       [0, 0, 0, 4, 0, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 6]])

In [ ]:
t = x
for c in range(A.shape[1]):
    v = A[:,c].T
    v = v.reshape(v.size,1)
    t = t - v * np.dot(v.T, t)[0][0]/np.dot(v.T,v)
    print('dot product',np.dot(v.T,t)[0][0])

In [ ]:
linalg.norm(x)

In [ ]:
s = 0
for a in x:
    s += a[0]*a[0]
np.sqrt(s)

In [ ]:
W[:,0]

In [ ]:
r